In [11]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
# from xiao_utils import months_among, f, f1
from xiao_utils import f

In [12]:
# 将level_id字段中的-替换为np.nan
df = pd.read_csv('../../data/origin/[new] yancheng_train_20171226.csv', dtype={'sale_date':str}, na_values=['-'], low_memory=False)
df['sale_date']= pd.to_datetime(df['sale_date'], format='%Y%m')

# 将price_level字段转换成有序类别的类型，并用其数值填入该列。
df['price_level'] = df['price_level'].astype('category', categories=['5WL','5-8W','8-10W','10-15W','15-20W','20-25W','25-35W','35-50W','50-75W'], ordered=True)
df['price_level'] = df['price_level'].cat.codes

# 待选方案：先把power和扭矩字段带/的行复制一份，然后将新行里的销量清零，将原行和新行的power和扭矩字段的值分别赋为slash前后的值。
# 现行方案：先他娘的直接把slash和后面的值删掉。省得影响记录条数相关的统计量。
def process_power_and_torque(s):
    return s.split('/')[0]
df['power'] = df['power'].astype(str).apply(process_power_and_torque).astype(float) #[18600]
df['engine_torque'] = df['engine_torque'].astype(str).apply(process_power_and_torque).astype(float)

# -------------------------------------------------------------
# 把2017年11月的数据拼进来，一块填入其特征，用于最终输出要提交的结果。
empty_Nov = pd.read_csv('../../data/origin/yancheng_testA_20171225.csv', dtype={'predict_date':str}, na_values=['-'], low_memory=False)
empty_Nov['predict_date']= pd.to_datetime(empty_Nov['predict_date'], format='%Y%m')
empty_Nov.rename(columns = {'predict_date': 'sale_date', 'predict_quantity':'sale_quantity'}, inplace = True)


# 读取玩了，先不急着拼，先把车型到品牌的映射关系join进来
class_to_brand = df[['class_id','brand_id']].groupby(['class_id']).mean().reset_index()
empyt_Nov = pd.merge(left=empty_Nov, right=class_to_brand, on='class_id', how='left')
empty_Nov['brand_id']= class_to_brand['brand_id']
# empty_Nov
# class_to_brand

# class_to_brand

# 读取完了，拼上去
df = pd.concat([df, empty_Nov])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20297 entries, 0 to 139
Data columns (total 32 columns):
TR                       20157 non-null object
brand_id                 20297 non-null int64
car_height               20157 non-null float64
car_length               20157 non-null float64
car_width                20157 non-null float64
class_id                 20297 non-null int64
compartment              20157 non-null float64
cylinder_number          20157 non-null float64
department_id            20157 non-null float64
displacement             20157 non-null float64
driven_type_id           20157 non-null float64
emission_standards_id    20157 non-null float64
engine_torque            20138 non-null float64
equipment_quality        20157 non-null float64
front_track              20157 non-null float64
fuel_type_id             20154 non-null float64
gearbox_type             20157 non-null object
if_MPV_id                20157 non-null float64
if_charging              20157 non-

In [13]:
%qtconsole

9. 过去几年中，当前月的总销量占当年总销量的比例
1. 过去几年中，单月各车型总销量

### 该类型特征的 column name 都以 T_ 开头，意为Time（时间）
#### 单纯的时间信息
1. 年
2. 月
3. 年、月的Onehot
4. 月份相关的【还未加入】
    4. 当月有几个周六
    5. 当月有几个周日
    6. 当月有多少天
    7. 当月周六+周日一共有多少天
    8. 当月周末占全月天数的比例（即上一条除以上上一条）
    10. 当月有多少天假期
9. 当月是哪个季节？？【还未加入】
1. 当月是上半年还是下半年？？【还未加入】

#### 历史销量信息（不区分车型/品牌）~~【还未加入】~~
1. 过去5年，单月各车型总销量【T_som_i, Sale of One Month】
3. 从去年开始往前看，当前月对应月份的销量在当年总销量中的比例（如当前行的sale_date=201710，则这个特征计算的是201610的总销量在2016年全年销量中的占比）【还未加入】


In [3]:
df.columns

Index(['TR', 'brand_id', 'car_height', 'car_length', 'car_width', 'class_id',
       'compartment', 'cylinder_number', 'department_id', 'displacement',
       'driven_type_id', 'emission_standards_id', 'engine_torque',
       'equipment_quality', 'front_track', 'fuel_type_id', 'gearbox_type',
       'if_MPV_id', 'if_charging', 'if_luxurious_id', 'level_id',
       'newenergy_type_id', 'power', 'price', 'price_level', 'rated_passenger',
       'rear_track', 'sale_date', 'sale_quantity', 'total_quality', 'type_id',
       'wheelbase'],
      dtype='object')

In [9]:
g_date = df[['sale_date','sale_quantity']].groupby('sale_date').sum()
g_date = g_date.rename(columns={'sale_quantity':'T_som_0'})


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71 entries, 2012-01-01 to 2017-11-01
Data columns (total 1 columns):
T_som_0    70 non-null float64
dtypes: float64(1)
memory usage: 1.1 KB


In [10]:
g_date.head()

,T_som_0
sale_date,
2012-01-01,22927.0
2012-02-01,14433.0
2012-03-01,15587.0
2012-04-01,13438.0
2012-05-01,18195.0


In [5]:
# 单纯时间信息
df['year'] = df['sale_date'].apply(lambda x:x.year)
df['month'] = df['sale_date'].apply(lambda x:x.month)
df['is_leap_year'] = df['sale_date'].apply(lambda x:x.is_leap_year)
df['year_oh'] = df['year']
df['month_oh'] = df['month']
df = pd.get_dummies(df, columns=['year_oh','month_oh'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20297 entries, 0 to 139
Data columns (total 53 columns):
TR                       20157 non-null object
brand_id                 20297 non-null int64
car_height               20157 non-null float64
car_length               20157 non-null float64
car_width                20157 non-null float64
class_id                 20297 non-null int64
compartment              20157 non-null float64
cylinder_number          20157 non-null float64
department_id            20157 non-null float64
displacement             20157 non-null float64
driven_type_id           20157 non-null float64
emission_standards_id    20157 non-null float64
engine_torque            20138 non-null float64
equipment_quality        20157 non-null float64
front_track              20157 non-null float64
fuel_type_id             20154 non-null float64
gearbox_type             20157 non-null object
if_MPV_id                20157 non-null float64
if_charging              20157 non-

In [7]:
T_features = pd.merge(df, g_date, how='left', left_on='sale_date', right_index=True)

In [8]:
T_features.to_csv("../../data/features/T_features.csv",index=False)

In [ ]:
# 历史销量信息
# 主要逻辑
def calc_sale_features_on_time(df):
    """
    Args:
        df: 完整的数据集
    Return:
        tmp：基于历史销量（不分品牌和车型）信息，构造出的特征们
    """
#     g_date = df.groupby(['class_id','sale_date']).sum().reset_index()[['class_id','sale_date','sale_quantity']]
#     gg = g_date.groupby('class_id')
    
    g_date = df[['sale_date','sale_quantity']].groupby('sale_date').sum()
    

    # 过去几年内的每个月销量
    tmp = g_cls_date
    for i in range(61):
        tmp['sale_of_month_' + str(i+1) + '_ago'] = gg.apply(f, -(i+1), -i).reset_index()['sale_date']


    # 该车型过去2~60个月分别的销量和
    tmp['sum_sale_of_last_1_month'] = tmp['sale_of_month_1_ago']
    for i in range(60):
        # tmp['sum_sale_of_last_' + str(i+1) + '_month'] = gg.apply(f, -(i+1), 0).reset_index()['sale_date']
        tmp['sum_sale_of_last_' + str(i+2) + '_month'] = tmp['sum_sale_of_last_' + str(i+1) + '_month'] + \
                                                            tmp['sale_of_month_' + str(i+2) + '_ago']
    tmp = tmp.drop('sum_sale_of_last_1_month', axis=1) # 再把这一列删掉，因为和前面的 sale_of_month_1_ago 列是重复的
    
    # 一阶差分
    for i in range(60):
        thismonth = tmp['sale_of_month_' + str((i+1)*12) + '_ago']
        lastmonth = tmp['sale_of_month_' + str((i+1)*12+1) + '_ago'] # gg.apply(f, -(i+1)*12-1, -(i+1)*12).reset_index()['sale_date']
        tmp['rate_of_this_month_divby_last_month_' + str(i+1) + '_year_ago'] = thismonth / lastmonth
        tmp['diff_of_this_month_sub_last_month_' + str(i+1) + '_year_ago'] = thismonth - lastmonth

    # 该车型往年这个月比上个月的销量比值
    # 该车型往年这个月减去上个月的销量差值
    for i in range(3): # 只看过去三年的
        thismonth = tmp['sale_of_month_' + str((i+1)*12) + '_ago']
        lastmonth = tmp['sale_of_month_' + str((i+1)*12+1) + '_ago'] # gg.apply(f, -(i+1)*12-1, -(i+1)*12).reset_index()['sale_date']
        tmp['rate_of_this_month_divby_last_month_' + str(i+1) + '_year_ago'] = thismonth / lastmonth
        tmp['diff_of_this_month_sub_last_month_' + str(i+1) + '_year_ago'] = thismonth - lastmonth

    # 该车型上个月比上上个月的比值
    thisyear_lastmonth = tmp['sale_of_month_1_ago']
    thisyear_lastlastmonth = tmp['sale_of_month_2_ago']
    tmp['rate_of_last_divby_lastlast'] = thisyear_lastmonth / thisyear_lastlastmonth
    # 该车型上个月减去上上个月的差值
    tmp['diff_of_last_sub_lastlast'] = thisyear_lastmonth - thisyear_lastlastmonth

    # 注意要把np.inf替换为空值，在上面算月销量比例时，引入了inf，其实应该作为空值。
    return tmp.replace([np.inf, -np.inf], np.nan)